# Наименование проекта
**Исследование объявлений о продаже квартир**

В нашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. 

## Задача
установить параметры, что позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

## Описание данных
По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных (например, расстояние до центра, аэропорта, ближайшего парка и водоёма)


`airports_nearest` — расстояние до ближайшего аэропорта в метрах (м) \
`balcony` — число балконов \
`ceiling_height` — высота потолков (м) \
`cityCenters_nearest` — расстояние до центра города (м) \
`days_exposition` — сколько дней было размещено объявление (от публикации до снятия) \
`first_day_exposition` — дата публикации \
`floor` — этаж \
`floors_total` — всего этажей в доме \
`is_apartment` — апартаменты (булев тип) \
`kitchen_area` — площадь кухни в квадратных метрах (м²) \
`last_price` — цена на момент снятия с публикации \
`living_area` — жилая площадь в квадратных метрах (м²) \
`locality_name` — название населённого пункта \
`open_plan` — свободная планировка (булев тип) \
`parks_around3000` — число парков в радиусе 3 км \
`parks_nearest` — расстояние до ближайшего парка (м) \
`ponds_around3000` — число водоёмов в радиусе 3 км \
`ponds_nearest` — расстояние до ближайшего водоёма (м) \
`rooms` — число комнат \
`studio` — квартира-студия (булев тип) \
`total_area` — общая площадь квартиры в квадратных метрах (м²) \
`total_images` — число фотографий квартиры в объявлении

# <a id="title11">План исследования:</a>

[**1. Открытие файла с данными и изучение общей информации**](#title1)\
[**2. Предобработка данных:**](#title2)\
           [2.1. Изучение и заполнение пропусков](#title2.1)\
           [2.2. Изучение типов данных](#title2.2)\
           [2.3. Изучение и обработка аномалий](#title2.3)\
           [2.4. Изучение и обработка дубликатов](#title2.4)\
           [2.5. Добавление в таблицу доп. столбцов для исследовательского анализа](#title2.5)\
[**3. Исследовательский анализ данных:**](#title3)\
            [3.1. Связь числа предложений и различных параметров квартиры](#title3.1)\
            [3.2. Связь стоимости/цены 1 кв.м и различных параметров квартиры](#title3.2)\
            [3.3. Топ-10 населённых пунктов датафрейма по объёмам продаж](#title3.3)\
            [3.4. Связь цены 1 кв.м. и удалённости квартиры от центра Санкт-Петербурга](#title3.4)\
[**4. Общий вывод**](#title4)

## <a id="title1">1. Открытие файла с данными и изучение общей информации.</a>

In [1]:
#  импортируем основные библиотеки
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#  читаем файл с локальной машины или с удалённого сервера (метод try - except)
try:
    data = pd.read_csv("/real_estate_data.csv", sep='\t')
except:
    data = pd.read_csv("https://code.s3.yandex.net/datasets/real_estate_data.csv", sep='\t')

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [ ]:
# устанавливаем отображение полного содержимого ячеек
pd.set_option('display.max_rows', 25)

*После загрузки датафрейма получаем предварительную информацию о характере данных*

In [ ]:
data.info()

In [ ]:
data.describe().T

In [ ]:
data.head(10)

In [ ]:
data.tail(10)

*Дополнительно изучим гистограммы по числовым данным*

In [ ]:
data.hist(figsize=(15, 15));

**Выводы:**

- полученный массив содержит данные различных типов, числовые и категориальные
- набор данных отличается неполнотой, в нём имеются явные аномалии в значениях, несоответствие типов данных и т.п.
- прежде, чем мы сможем проанализировать данные и сделать выводы о параметрах, необходимо произвести преобработку

## <a id="title2">2. Предобработка данных</a>
[к оглавлению](#title11)

### <a id="title2.1">2.1.Изучение и заполнение пропусков в данных</a>

In [ ]:
#изучим число пропусков в каждом из столбцов датафрейма
data.isnull().sum()

**Выводы**
- налицо большое (от 20 до 50%) число пропусков во всех картографических данных, а также ряд пропусков в заполненных пользователем данных: `ceiling_height`, `living_area`, `is_apartment`, `kitchen_area`, etc...

Пропуски в картографических данных возможны по следующим причинам:
- слияние данных из разных датафреймов, для одного (или нескольких) из которых измерения расстояний и расчёт числа объектов не осуществлялись и как результат - их нет в итоговом датафрейме, который мы рассматриваем
- сбой при объединении (например, из-за различия в указанных единицах измерения)

Пропуски в "пользовательских" данных возможны по следующим причинам:
- заполнение анкет без документов о собственности на руках, в результате чего ряд "незапоминающихся" параметров (высота потолков, жилая площадь и площадь кухни) не были указаны
- реализация выбора ("апартаменты"/"не апартаменты") через чекбоксы, заполнение которых не обязательно для подачи анкеты, как результат - только 10% пользователей указали этот параметр
- "по умолчанию" число балконов не объявлялось, как результат - владельцы квартир без балконов не заполнили этот пункт
- отсутствие данных в пункте о полном числе этажей и наименовании населённого пункта может быть вызвано техническими сбоями при переносе (и/или объединении) данных

**Предполагаемые способы коррекции недостающих сведений в датафрейме:**

- `locality_name` - можно заполнить по моде столбца (но это может повлечь за сообой ошибки), корректнее заполнить однородными значениями (напр. Unknown) /или удалить строки/
- `balcony` - предположительно, пропуски в данных указывают на отсутствие балконов в квартире, заполним пропуски 0
- `is_apartment` - важный критерий при выборе квартиры, но в дальнейшем анализе данных не задействован, можем как удалить столбец, так и заполнить пропуски преобладающим значением (предположительно `False`)
- `days_exposition` - пропуски в этом столбце могут быть вызваны тем, что на момент выгрузки данных жилой объект ещё не был снят с продажи/продан, изучим характер и распределение пропусков, после чего решим, каким образом их заполнять
- `floors_total` - учитывая незначительное (~0,3% от общего набора данных) число пропусков (и как результат - незначительное влияние на итоговый анализ) удалим строки с пропущенными в этом столбце значениями
- `ceiling_height` - так как параметр стандартизуется (и, исходя из общих соображений, находится в диапазоне 2,4-4 м), - заполним пропуски значением медианы, как наиболее устойчивой к выбросам и отклонениям величиной
- `living_area`, `kitchen_area` - нестандартизуемые параметры, попробуем поискать закономерности в соотношении площадей в квартирах различной total_area, если получится, то заполним пропущенные значения на этом основании(или оставим пропуски в данных)
`airports_nearest` - возможно заполнение пропусков по медиане для каждого конкретного населённого пункта (кроме уникальных значений*)
`cityCenters_nearest`, `parks_around3000`, `parks_nearest`, `ponds_around3000`, `ponds_nearest` - картографические данные, нет возможности самостоятельно произвести необходимые измерения, вероятно, придётся оставить пропуски в этих данных незаполненными

При заполнении пропусков, будем выводить на экран сведения о числе пропусков в столбце до и после заполнения

    *В случае уникальных значений (т.е. когда объект с незаполненными данными - единственный в своём населённом пункте), при их сравнительно малом количестве оставим такие ячейки без заполнения

*Заполним отсутствующие в `locality_name` значения на "Unknown"*

In [ ]:
print('Пропуски до внесения изменений:', data['locality_name'].isna().sum())
data['locality_name'].fillna('Unknown', inplace=True)
print('Пропуски после внесения изменений:', data['locality_name'].isna().sum())

*Заполним нулями отсутствующие значения в `balcony`*

In [ ]:
print('Пропуски до внесения изменений:', data['balcony'].isna().sum())

data['balcony'].fillna(0, inplace=True)

print('Пропуски после внесения изменений:', data['balcony'].isna().sum())

*Определим моду (среди заполненных значений) в столбце `is_apartment`*

In [ ]:
data['is_apartment'].value_counts()

*Заполним модой пропущенные значения для этого столбца*

In [ ]:
print('Пропуски до внесения изменений:', data['is_apartment'].isna().sum())

data['is_apartment'].fillna(False, inplace=True)

print('Пропуски после внесения изменений:', data['is_apartment'].isna().sum())

In [ ]:
print('Пропуски до внесения изменений:', data['floors_total'].isna().sum())

In [ ]:
data.dropna(subset=['floors_total'],inplace=True)

In [ ]:
print('Пропуски после внесения изменений:', data['floors_total'].isna().sum())

*После удаления строк переустановим индекс в датафрейме*

In [ ]:
data.reset_index(drop=True, inplace= True)

*Изучим столбец `days_exposition`:*

базовая гипотеза - пропущенные значения в нём соответствуют сравнительно недавнему размещению объявления и жилой объект на момент выгрузки данных ещё не был продан (или объявление с ним не было удалено из базы)*

*Сперва преобразуем значения в столбце `first_day_exposition` в формат DateTime для удобства работы с данными*

In [ ]:
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format="%Y-%m-%dT%H:%M:%S")

*Построим гистаграмму зависимости числа NaN значений в столбце `days_exposition` от значений в столбце `first_day_exposition`*

In [ ]:
data.query('days_exposition.isna()')['first_day_exposition'].hist(bins=20, figsize=(8,5));

Гипотеза подтверждается: основная масса объявлений, по которым отсутствует срок нахождения в базе, размещены сравнительно недавно, менее года (на момент последнего объявления/выгрузки данных). Чтобы искусственно не изменять число проданных/снятых с размещения квартир, оставим пропуски в этом столбце незаполненными, оставив при этом сведения о квартирах в датафрейме*

*Заполняем пропуски в столбцах `airports_nearest`, `cityCenters_nearest`, `ceiling_height` медианными значениями (используя методы `groupby` и `transform`):*

In [ ]:
print('Пропуски до внесения изменений:', data['airports_nearest'].isna().sum())

In [ ]:
data['airports_nearest'] = data['airports_nearest'].fillna(data.groupby('locality_name')['airports_nearest'].transform('median'))

In [ ]:
print('Пропуски после внесения изменений:', data['airports_nearest'].isna().sum())

**Вывод:**
Заполнение пропусков расстояния до аэропорта (по медиане для каждого населённого пункта) не позволило существенно сократить число NaN ячеек, вероятно, из-за большого количества населённых пунктов, для которых расчёт такого расстояния не был произведён вовсе. При дальнейшем анализе учтём это.

In [ ]:
print('Пропуски до внесения изменений:', data['cityCenters_nearest'].isna().sum())

In [ ]:
data['cityCenters_nearest'] = data['cityCenters_nearest'].fillna(data.groupby('locality_name')['cityCenters_nearest'].transform('median'))

In [ ]:
print('Пропуски после внесения изменений:', data['cityCenters_nearest'].isna().sum())

**Вывод:**
Заполнение пропусков расстояния до центра города (по медиане для каждого населённого пункта) не позволило существенно сократить число NaN ячеек, вероятно, из-за большого количества населённых пунктов, для которых расчёт такого расстояния не был произведён вовсе. При дальнейшем анализе учтём это.

In [ ]:
print('Пропуски до внесения изменений:', data['ceiling_height'].isna().sum())

In [ ]:
data['ceiling_height'] = data['ceiling_height'].fillna(data.groupby('locality_name')['ceiling_height'].transform('median'))

In [ ]:
print('Пропуски после внесения изменений:', data['ceiling_height'].isna().sum())

**Вывод:**
заполнение по медиане качественно сработало для пользовательских данных и не очень помогло с картографическими данными (видимо, в связи с недостатком соответствующих сведений о многих населённых пунктах в области => невозможностью расчёта для них медианы). Так как пока нет понимания, каким образом заполнять эти данные, оставим их без изменений.

*Недостающие сведения в пользовательских данных (`ceiling_height`) заполним медианными значениями (рассчитанными для всего столбца)*

In [ ]:
data['ceiling_height'].fillna(data['ceiling_height'].median(), inplace=True)

*Проверим весь датафрейм на корректное заполнение пропусков, методы `isnull` и `sum`*

In [ ]:
data.isnull().sum()

**Вывод:**
Мы заполнили пропуски во всех пользовательских данных (кроме пропусков в площадях кухни и жилой), ряд сведений в картографических данных придётся оставить без заполнения из-за невозможности в рамках проекта получить дополнительные сведения для заполнения

### <a id="title2.2">2.2. Изучение типов данных</a>
[к оглавлению](#title11)

In [ ]:
data.dtypes

Ряд данных, представленных в формате float может быть (по смыслу и для корректности сравнения в дальнейшем) переведён в integer (число этажей и балконов, цена продажи). 

*Заменим тип данных, используя метод `astype` и проверим корректность преобразования*

In [ ]:
# формируем словарь для одновременной замены типов данных в разных столбцах
convert_dict = {'last_price': int,
               'floors_total': int,
               'balcony': int               
               }
data = data.astype(convert_dict) #  меняем типы данных с помощью словаря

In [ ]:
data.dtypes #  проверим корректность преобразования типов данных

Преобразование типов данных осуществлено корректно. Логичным было бы преобразовать в формат `integer` и столбцы с данными о числе близлежащих парков и прудов, число дней экспозиции, но, в связи с сохранившимися в них пропусками, такая операция пока невозможна

### <a id="title2.3">2.3. Изучение и обработка аномалий в данных</a>
[к оглавлению](#title11)

*Для формирования максимально корректного датафрейма для последующего анализа, изучим ряд столбцов по признаку:*
1) в них могут содержаться аномальные значения &
2) мы будем использовать их при дальнейшем анализе

In [ ]:
data[['rooms', 'ceiling_height', 'total_area', 'living_area', 'kitchen_area', 'last_price']].describe().T

**Вывод:**
 - аномальные (необычные) данные присутствуют в столбцах `rooms` (0 число комнат), `ceiling_height` (min и max высота потолка нелогичны для квартир)
 - проверим `...area` столбцы, max показатели кажутся завышенными в 10 раз, а min - заниженными
 - проверим и данные по `last_price` - в этом столбце слишком низкая min цена и необычно высокая max*

*Сперва изучим 0 значения в столбце `rooms`:* \
Предположим, что в квартирах со свободной планировкой комната и кухня как правило объединены, поэтому число комнат в анкете для таких квартир пользователь мог указать равным 0

In [ ]:
#  изучим параметры столбца "open_plan" для квартир с 0 комнат
data.query('rooms == 0')['open_plan'].value_counts() 

In [ ]:
# проверим общее число квартир со свободной планировкой
data['open_plan'].value_counts() 

Гипотеза о том, что для квартир со свободной планировкой чаще всего указывается число комнат = 0, подтверждается.

*Основываясь на этом, заполним столбец `rooms` для всех квартир (со свободной планировкой) число комнат = 0*

In [ ]:
data.loc[(data['open_plan'] == True), 'rooms'] = 0

*Теперь проверим, есть ли квартиры с 0 комнат, без свободной планировки, при этом не являющиеся студиями*

In [ ]:
data.query('rooms == 0 and open_plan != True and studio != True').groupby(by=['total_area']).first()

**Вывод:**
*Все квартиры с 0 в столбце `rooms` являются либо студиями, либо квартирами со свободной планировкой, для которых нулеое число комнат  - не аномальное значение, оставим такие данные*

Перейдём к разбору аномальных значений в столбце 'ceiling_hight'. \
*Для начала посмотрим диапазон возможных значений на гистограммах (с разными range, в диапазоне от min до max)*

In [ ]:
data['ceiling_height'].hist(bins=10, range=(1,2), figsize=(4,3));
plt.show()
data['ceiling_height'].hist(bins=20, range=(2,5), figsize=(4,3));
plt.show()
data['ceiling_height'].hist(bins=50, range=(5,100), figsize=(4,3));
plt.show()

Видим, что подавляющее большинство значений лежит в диапазоне 2.4 - 4 метра. \
С единичными ошибками при заполнении (высота менее 2.4, в диапазоне от 4 и до 24 м и более 40 м) мы ничего не сумеем сделать, так как непонятно, где именно пользователь допустил ошибку при заполнении. \
*Для диапазона значений от 24 и до 40 м сделаем предположение, что пользователь при заполнении забыл указать знак запятой и разделим указанную в df высоту потолков на 10*

In [ ]:
data.loc[(data['ceiling_height'] >= 24), 'ceiling_height'] = \
data.loc[(data['ceiling_height'] >= 24), 'ceiling_height'] / 10  #  #  все высоты потолков выше 24 м уменьшаем в 10 раз

*Посчитаем число строк, в которых значения "высота потолков" выходят за границы "нормального" диапазона 2.5-4 м (т.е. "аномальны")*

In [ ]:
data.query('ceiling_height < 2.5 or ceiling_height > 4')['ceiling_height'].count()

**Вывод**
Число строк с аномальными данными в столбце `ceiling_height` < 0,4% от общего набора, их влиянием в ходе дальнейшего анализа можно пренебречь
*Исключим строки с этими данными из общего датафрейма, после чего переустановим индекс датафрейма*

In [ ]:
data = data.query('ceiling_height >= 2.5 and ceiling_height <= 4')

In [ ]:
data.reset_index(drop=True, inplace= True)

*Изучим аномальные значения в столбцах `..._area`:*
1. Исходя из соображения, что (жилая площадь + площадь кухни) <= общей площади квартиры, будем искать строки, где это правило не выполняется (предполагаем ошибку при заполнении):

In [ ]:
data.query('(total_area + kitchen_area) < living_area')

*Таких аномалий в заполнении нет (или они были исключены на предыдущих этапах удаления строк*

2. Исходя из соображения, что, как правило, жилая площадь > площади кухни, найдём строки, где это правило не выполняется (предполагаем ошибку при заполнении):

In [ ]:
data.query('living_area < kitchen_area')

Обнаружили несколько сот строк, где, возможно, при вводе данных пользователем были перепутаны друг с другом значения в столбцах kitchen_area и living_area.\
*Но гарантировать, что это именно опечатка пользователя, а не квартира варианта планировки 2Е (1 комната + кухня-гостиная) мы не можем, поэтому не считаем эти значения аномальными, изменений не вносим*

3. При заполнении пользователи могли опечаткой увеличить/уменьшить разряд данных, проверим, есть ли в датафрейме строки, где соотношение общей и жилой площади отличаются на порядок:

In [ ]:
data.query('total_area / 10 > living_area')

*Поправим данные с явно заниженным (из-за опечатки) размером жилой площади:*

In [ ]:
#  правим данные в столбце `living_area` с заниженной в 10 раз жилой площадью 
data.loc[(data['living_area'] < 6) & (data['living_area'] * 10 < data['total_area']), 'living_area'] = \
data.loc[(data['living_area'] < 6) & (data['living_area'] * 10 < data['total_area']), 'living_area'] * 10

*Изучим аномальные значения в столбце `last_price`, начав со сверхдорогих предложений:*

In [ ]:
data.query('last_price > 50000000').groupby('last_price')['total_area'].mean()

**Выводы**

- число "сверхдорогих" (стоимость от 50 млн.руб) квартир измеряется десятками, т.е. занимает незначительную долю в общем массиве исследуемых данных
- как правило, это квартиры с большой площадью, число таких уникальных квартир и не должно быть слишком большим в датафрейме со стандартными предложениями о продаже. Поэтому данные о сверхдорогих квартирах (несмотря на возможное наличие опечаток в цене) никак корректировать не будем. При необходимости, в дальнейшем сможем отфильтровать явные аномалии, изучив сведения о стоимости квадратного метра.
- согласно данным ЦИАН, максимальная площадь продаваемого как единая квартира объекта в Санкт-Петербурге составляет ~ 940 кв.м, поэтому данные о квартирах сопоставимой площади исключать из датафрейма не будем

*Теперь посмотрим на "сверхдешёвые" квартиры:* \
Данные по Санкт-Петербургу и области лучше смотреть отдельно, примем за критерий "дешевизны" стоимость до 1 млн.руб для области и до 1,5 млн.руб для СПб)

In [ ]:
data.query('last_price <= 1000000 and locality_name != "Санкт-Петербург"').groupby(['last_price']).first().head(10)

**Вывод:**

- минимальная цена фигурирует в объявлениях о продаже квартир в деревнях и сёлах Ленинградской области, 
- не зная статистики и положения дел в указанных населённых пунктах сложно сделать вывод, аномальны ли эти данные (низкая цена указана только для привлечения внимания, опечатка и т.п.)
- оставим эти данные без изменений

*Теперь посмотрим на данные о min ценах по Санкт-Петербургу:*

In [ ]:
data.query('last_price <= 1500000 and locality_name == "Санкт-Петербург"')

**Выводы:**
- число таких квартир невелико, судя по срокам продажи, можно предположить, что низкая цена была вызвана необходимостью срочной продажи или иными,  не учитываемыми факторами
- сильно выделяется одно значение, по порядку цены видно, что при публикации цена была указана в тыс. руб. 

*Скорректируем эту цену вручную*

In [ ]:
data.loc[(data['last_price'] < 20000),'last_price'] = data.loc[(data['last_price'] < 20000),'last_price'] * 1000

*Изучим аномалии в данных по жилой площади (исходя из данных https://realty.rbc.ru/ о том, что минимальная общая площадь предлагаемых в Санкт-Петербурге и области квартир составляет 18 кв.м)*

In [ ]:
data.query('total_area < 18 and studio == False')

Таких квартир чуть больше десятка \
*Отфильтруем в датафрейм только квартиры с полной площадью более 17 кв.м, после чего переустановим индекс*

In [ ]:
data = data.query('total_area > 17 or studio == True')

In [ ]:
data.reset_index(drop=True, inplace= True)

*Изучим квартиры с площадью кухни менее 4 кв.м*

In [ ]:
data.query('kitchen_area < 4 and open_plan != True')

Таких квартир несколько десятков. Остальные данные (по жилой и общей площади в частности) сомнений не вызывают. И хотя столь малые размеры кухни (1.3 кв.м) аномальны, но также вполне могут быть результатом произведённой перепланировки квартир, когда оставлена лишь "кухонная ниша" \
*Оставим их в датафрейме*

*Изучим квартиры с жилой площадью менее 10 кв.м*

In [ ]:
data.query('living_area < 10')

Таких квартир пара десятков. Остальные данные (по общей площади в частности) аномальными не выглядят. И хотя столь малые площади кажутся следствием опечатки и выглядят, как аномальные, но также вполне могут быть результатом произведённой перепланировки квартир, например, с модификацией 2E, упомянутой выше \
*Оставим их в датафрейме*

*Изучим квартиры, находящиеся ближе 3 км от аэропорта*

In [ ]:
data.query('airports_nearest <= 3000')

*Такая квартира всего одна, и хотя в остальных данных аномалий не видно, но расстояние в 0 км от аэропорта явно ошибочно, можно отфильтровать сведения о ней из датафрейма*

In [ ]:
data = data.query('airports_nearest != 0')

In [ ]:
data.reset_index(drop=True, inplace= True)

*Изучим на аномалии данные о полном числе этажей (исходя из имеющейся в интернете информации, что максимальный по количеству этажей жилой дом в Санкт-Петербурге и области - 36-этажный)*

In [ ]:
data.query('floors_total > 36')

Таких значений всего 2, понять, каким образом была допущена опечатка, мы не можем \
*Удалим эти значения из датафрейма*

In [ ]:
data = data.query('floors_total <= 36')

In [ ]:
data.reset_index(drop=True, inplace=True)

*Помотрим данные по итогам удаления аномалий*

In [ ]:
data.describe().T

**Выводы:**

Предварительный анализ аномалий в данных завершён:
- ряд явно аномальных значений был обнаружен и строки, содержащие такие значения (суммарно ~ 1% данных) из датафрейма были удалены
- мы убедились в том, что полностью избавиться от аномальных значений не получится в связи с неопределённостью данных, и, как следствие, невозможностью правки и при этом сомнительностью удаления (напр. очень малые min площади кухни и жилой площади)
- при этом в общем массиве датафрейма оставшиеся "возможно-аномальные" значения составляют малую часть и не должны кардинально влиять на взаимозависимости данных. 
- возможно, часть аномалий удастся "отловить" и удалить на следующем этапе, после изучения доп. параметров (цена кв. метра и т.п.)

### <a id="title2.4">2.4. Изучение и обработка дубликатов</a>
[к оглавлению](#title11)

*С целью избавления от возможных дубликатов в данных, изучим столбец с наименованием населённых пунктов `locality_name`*

In [ ]:
sorted(data['locality_name'].unique())

In [ ]:
# расчёт числа уникальных значений
len(data['locality_name'].unique())

Cреди нескольких сотен уникальных значений используются разнообразные определения для посёлков (поселок, посёлок городского типа и т.д.) и СНТ (садоводческое некоммерческое товарищество, садовое товарищество и т.д. )

*Для устранения возможных дубликатов, сперва удалим разнообразие таких наименований, оставив только названия населённых пунктов, используем для замены метод `str.replace`*

In [ ]:
#  создадим словарь (для удаления значений на его основе)
pattern = ['деревня ', 'поселок ','городского типа ', 'городской', 'посёлок ', 'при железнодорожной ', 'село ',
           'станции ', 'садоводческое некоммерческое товарищество ', 'садовое товарищество ', 'коттеджный ']

#  в цикле удалим наименования типов населённых пунктов в столбце 'locality_name' и пробелы, остающиеся после удаления
for value in pattern:
    data['locality_name'] = data['locality_name'].str.replace(value, '').str.lstrip()

In [ ]:
len(data['locality_name'].unique())

In [ ]:
sorted(data['locality_name'].unique())

**Вывод:**
Мы привели различные наименования населённых пунктов к единообразию, очистка столбца `locality_name` от возможных дубликатов произведена (за исключением возможных опечаток в наименованиях населённых пунктов)

*Проверим предобработанный датафрейм на дубликаты, для чего создадим временный датафрейм (в котором приведём к единому (нижнему) регистру наименования населённых пунктов в столбце `locality_name`) и применим к датафрейму методы `duplicated` и `sum`*

In [ ]:
data_temp = data

In [ ]:
data_temp['locality_name'].str.lower()

In [ ]:
data_temp.duplicated().sum()

*Явные дубликаты не обнаружены, на этом предварительную обработку данных можно считать завершённой*

### <a id="title2.5">2.5. Добавление в таблицу доп. столбцов для исследовательского анализа</a>
[к оглавлению](#title11)

- `sqmeter_price`- цена одного квадратного метра
- `publication_week_day` - день публикации объявления (0 - monday, 1 - friday etc.)
- `publication_month` - месяц публикации объявления
- `publication_year`- год публикации объявления
- `floor_type` - тип этажа квартиры («first»(1), «other»(2), «last»(3))
- `km_to_cityCenter` - расстояние в км до центра города

In [ ]:
#  формируем столбец с ценой одного квадратного метра (округлённой)
data['sqmeter_price'] = round(data['last_price'] / data['total_area'])

In [ ]:
#  формируем столбец с днём недели, когда было опубликовано объявление
data['publication_week_day'] = data['first_day_exposition'].dt.weekday  

In [ ]:
#  формируем столбец с номером месяца, в котором было опубликовано объявление
data['publication_month'] = data['first_day_exposition'].dt.month

In [ ]:
#  формируем столбец с годом, в котором было опубликовано объявление
data['publication_year'] = data['first_day_exposition'].dt.year

In [ ]:
#   для создания столбца с категорий этажа сперва зададим функцию категоризации, 
#   устанавливающую признак этажа в зависимости от его сравнения с общим числом этажей
#   для удобства построения графиков обозначим признак этажа цифрой: 1- первый, 2 - любой другой, 3 - последний

def floor_type(data_floor):
    if data_floor['floor'] == 1: return 1  
    elif data_floor['floor'] < data_floor['floors_total']: return 2
    elif data_floor['floor'] == data_floor['floors_total']: return 3

#  применим эту функцию к датафрейму, метод apply(), формируя требующийся столбец с категорией этажа

data['floor_type'] = data.apply(floor_type, axis=1)

In [ ]:
#  формируем столбец с расстоянием от центра нас. пункта в км (с точностью до 1 км)
data['km_to_cityCenter'] = round(data['cityCenters_nearest'] / 1000)  

**Вывод:**
Cформированы необходимые для анализа данных о продажах дополнительные столбцы



## <a id="title3">3. Исследовательский анализ данных</a>
[к оглавлению](#title11)

### <a id="title3.1">3.1. Связь числа предложений и различных параметров квартиры</a>

*При дальнейшем анализе нас будет интересовать связь между числом предложений и следующими параметрами:*
 
- общая площадь;
- жилая площадь;
- площадь кухни;
- цена объекта;
- количество комнат;
- высота потолков;
- этаж квартиры;
- тип этажа квартиры («первый», «последний», «другой»);
- общее количество этажей в доме;
- расстояние до центра города в метрах;
- расстояние до ближайшего аэропорта;
- расстояние до ближайшего парка;
- день и месяц публикации объявления
- срок размещения объявления
- цена квадратного метра

*На гистаграммах посмотрим распределение по интересующим нас параметрам*

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(10,4))
ax1.hist(data['total_area'],bins=20)
ax1.set(ylabel='Кол-во предложений', xlabel='Полная площадь, кв.м', title='Полный диапазон')
ax2.hist(data['total_area'], range=(10,150), bins=20)
ax2.set(xlabel='Полная площадь, кв.м', title='Диапазон 10-150 кв.м')
plt.show;

Общая площадь: подавляющее количество предложений в рассматриваемом датафрейме  -  в диапазоне 40-80 кв.м, это типичные варианты массовой застройки 1-3к квартир. Имеются также единичные предложения с площадями в несколько сотен кв.м (согласно информации в интернете, это возможно, такие квартиры на рынке СПб существуют). При дальнейшем анализе посмотрим на связь между этим параметром и ценой (полной и за кв.м)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(10,4))
ax1.hist(data['living_area'],bins=20)
ax1.set(ylabel='Кол-во предложений', xlabel='Жилая площадь, кв.м', title='Полный диапазон')
ax2.hist(data['living_area'], range=(5,100), bins=40)
ax2.set(xlabel='Жилая площадь, кв.м', title='Диапазон 5-100 кв.м')
plt.show;

Жилая площадь: подавляющее количество предложений - в диапазоне 15-35 кв. метров, это типичные варианты массовой застройки 1-3к квартир, с яркими пиками на 18 кв.м (площадь одной комнаты) и 35 кв.м (площадь двух комнат). Имеются также единичные предложения с площадями в несколько сот кв.м. При дальнейшем анализе посмотрим на связь между этим параметром и ценой (полной и за кв.м)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(10,4))
ax1.hist(data['kitchen_area'],bins=20)
ax1.set(ylabel='Кол-во предложений', xlabel='Площадь кухни, кв.м', title='Полный диапазон')
ax2.hist(data['kitchen_area'], range=(2,20), bins=40)
ax2.set(xlabel='Площадь кухни, кв.м', title='Диапазон 2-20 кв.м')
plt.show;

Площадь кухни: подавляющее количество предложений - в диапазоне 6-12 кв.м, это типовой вариант для основного, представленного в датафрейме масссива 1-3 комнатных квартир. Есть несколько сотен вариантов с заметно большими размерами кухни (и единичные - с сотнями кв. метров), возможно - результат перепланировки и уникальные предложения, соответствующие квартирам с чрезвычайно большой общей площадью. При дальнейшем анализе посмотрим на связь между этим параметром и ценой (полной и за кв.м)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(8,4))
ax1.hist(data['last_price'],bins=30)
ax1.set(ylabel='Кол-во предложений', xlabel='Цена, руб.', title='Полный диапазон')
ax2.hist(data['last_price'], range=(100000,15000000), bins=30)
ax2.set(xlabel='Цена, руб.', title='Диапазон 10-15000 тыс. руб.')
plt.show;

Цена объекта: основная часть предложений - в ценовом сегменте до 10 млн. руб), его имеет смысл рассмотреть подробнее, хотя есть и эксклюзивные предложения, с ценой в несколько сотен млн.руб. Далее в анализе оценим влияние различных параметров квартиры на её полную цену.

In [ ]:
data['rooms'].hist(bins=15, figsize=(6,4)).set(ylabel='Кол-во предложений', xlabel='Число комнат', 
                                               title='Связь числа комнат и количества предложений')
plt.show;

Число комнат: основной массив предложений, как и можно было предположить из анализа площадей квартир - это 1-3 к. квартиры + небольшое количество уникальных предложений, с числом комнат до 19. Далее проанализируем взаимосвязь между числом комнат и ценой квартиры (полной и за кв.м)

In [ ]:
data['ceiling_height'].hist(bins=20, figsize=(6,4)).set(ylabel='Кол-во предложений', xlabel='Высота потолков, м',
                                                       title='Связь высоты потолков и количества предложений')
plt.show;

Высота потолков: этот столбец был на этапе предобработки "нормализован", в нём не осталось аномальных значений, большинство квартир вписывается в "стандартные" высоты 2,4-2.8 метра, с явными пиками на высотах 2,5 и 2,7 м, характерными для типовой советской застройки. Несколько сот предложений большей высоты - это могут быть как квартиры в дореволюционном жилом фонде, так и новые, сделаные по специальным проектам, квартиры

In [ ]:
data['floor'].hist(bins=20, figsize=(6,4)).set(ylabel='Кол-во предложений', xlabel='Номер этажа',
                                              title='Связь номера этажа и количества предложений')
plt.show;

Этаж квартиры: основная часть предложений - на "невысоких", до 10го, этажах, более половины - на этажах до 5го включительно (что отражает состояние жилого фонда в Санкт-Петербурге, где высотное строительство началось сравнительно недавно и на момент формирования датафрейма не существовало массового предложения о продаже в высотных жилых комплексах)

In [ ]:
data['floor_type'].hist(bins=5, figsize=(6,4)).set(ylabel='Кол-во предложений', xlabel='Категория этажа',
                                                  title='Связь категории этажа и количества предложений')
plt.xticks([1,2,3], ['первый', 'другой', 'последний'])
plt.show()

Тип этажа квартиры («первый», «последний», «другой»): значительное число предложений по продаже - на "некрайнем" этаже, хотя "первый" + "последний" этажи также составляют порядка четверти от всех предложений, в дальнейшем анализе посмотрим взаимосвязь между "категорией" этажа и полной стоимостью / ценой за кв. м.

In [ ]:
data['floors_total'].hist(bins=20, figsize=(6,4)).set(ylabel='Кол-во предложений', xlabel='Число этажей в доме',
                                                     title='Связь числа этажей в доме и количества предложений')
plt.show()

Общее количество этажей в доме: основной массив объявлений - в домах низкой этажности (до 10), с характерными пиками на 5 и 9 этажах, есть локальные пики, указывающие на предложения в высотках

In [ ]:
data['cityCenters_nearest'].hist(bins=20, figsize=(6,4)).set(ylabel='Кол-во предложений', 
                            xlabel='Расстояние до центра населённого пункта, м',
                            title='Связь расстояния до центра нас. пункта и количества предложений')
plt.show()

(для трёх следующих столбцов при оценке данных надо помнить что около четверти значений в датафрейме отсутствует из-за возможных ошибок при переносе данных или отсутствия картографической информации о населённом пункте):

Расстояние до центра города в метрах: основной массив квартир расположен в пределах 20 км от центра города (что явно характерно для самого Санкт_Петербурга). Далее проанализируем зависимость ценовых параметров квартир от удаления от центра.

In [ ]:
data['airports_nearest'].hist(bins=20, figsize=(6,4)).set(ylabel='Кол-во предложений', xlabel='Расстояние до аэропорта, м',
                                                         title='Связь расстояния до аэропорта и количества предложений')
plt.show()

Расстояние до ближайшего аэропорта: основная часть предлагаемых квартир расположена равномерно в диапазоне 10-35 км. от аэропорта "Пулково", что соответствует общему условию - 3/4 квартир в датафрейме из Санкт-Петербурга

In [ ]:
data['parks_nearest'].hist(bins=20, figsize=(6,4)).set(ylabel='Кол-во предложений',
                                                       xlabel='Расстояние до ближайшего парка, м',
                                                      title='Связь расстояния до ближайшего парка и количества предложений')
plt.show()

Расстояние до ближайшего парка: хорошо виден пик (близкого к нормальному) распределения на отметке ~ 500м. Зелёные насаждения распространены и доступны в СПб и большей части ЛО

In [ ]:
data['publication_week_day'].hist(bins=13, figsize=(6,4)).set(ylabel='Кол-во предложений',
                                                              xlabel='День недели публикации объявления',
                                                             title='Связь дня недели публикации и количества предложений')
plt.xticks([0,1,2,3,4,5,6], ['пн', 'вт', 'ср', 'чт', 'пт', 'сб', 'вс'])
plt.show()

День публикации объявления: публикации распределены равномерно по рабочим дням, и их число заметно сокращается в выходные дни. Дальше в анализе проследим, связан ли день публикации объявления с стоимостными параметрами

In [ ]:
data['publication_month'].hist(bins=23, figsize=(6,4)).set(ylabel='Кол-во предложений', xlabel='Месяц публикации объявления',
                                                          title='Связь месяца публикации и количества предложений')
plt.xticks(range(1,13), ['янв', 'фев', 'мар', 'апр', 'май', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек'])
plt.show()

Месяц публикации: пик приходится на весенние и осенние месяцы, в то время как летом и в пред/посленовогодний период количество публикуемых объявлений сокращается. Далее проверим при анализе, есть ли корреляция между днём/месяцем публикации и сроком продажи квартиры (числом дней экспозиции), а также стоимостными параметрами

In [ ]:
data['days_exposition'].hist(bins=20, figsize=(6,4)).set(ylabel='Кол-во предложений', 
                                                         xlabel='Срок размещения объявлений, дней',
                                                        title='Связь срока размещения и количества предложений')
plt.show()

Срок размещения объявления: можно видеть, что квартиры продавались (или снимались с продажи) в массе своей в пределах 150 дней, что может быть связано с различными параметрами, в том числе и условиями публикации объявлений. Хоте некоторые предложения сохранялись в базе весь период (11.2014-05.2019)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(8,4))
ax1.hist(data['sqmeter_price'],bins=30)
ax1.set(ylabel='Кол-во предложений', xlabel='Цена 1 кв.м, руб.', title='Полный диапазон')
ax2.hist(data['sqmeter_price'], range=(10000,300000), bins=30)
ax2.set(xlabel='Цена 1 кв.м, руб.', title='Диапазон 10-300 тыс. руб.')
plt.show()

Цена 1 кв.м.: достаточно ожидаемое нормальное распределение этого параметра с пиком в районе 90-100 тыс.р. (и "хвост" из очень дорогих уникальных предложений)

*Изучим показатель `days_exposition`, указывающий на СРОК ПРОДАЖИ КВАРТИРЫ*

In [ ]:
data['days_exposition'].describe().T

*Построим гистаграммы и  boxplot по всему диапазону значений и в пределах (mean + std)*

In [ ]:
data['days_exposition'].hist(bins=50, range=(0,1620), figsize=(6,4)).set(title='Срок продажи квартиры (весь диапазон)', 
    xlabel='дней до продажи', ylabel='число объектов');
plt.show()
data['days_exposition'].hist(bins=50, range=(0,420), figsize=(6,4)).set(title='Срок продажи квартиры (mean + std)', 
    xlabel='дней до продажи', ylabel='число объектов');
plt.show()

In [ ]:
data.boxplot(column='days_exposition');
plt.show()
data.query('days_exposition < (days_exposition.mean() + days_exposition.std())').boxplot(column='days_exposition');
plt.show()

**Вывод:** 
большинство квартир в изучаемом датафрейме продалось в течение первого года, при этом видны заметные пики на ~45 и ~60 днях (возможно, связанных со стандартными периодами оформления продаж наиболее удачных предложений), что и привело к медианному значению срока продажи в 94 дня, почти в 2 раза отличающемуся от среднего значения, которое учитывает выбросы, с временем продажи в сроки от года и выше (до 5 лет!) \
Исходя из этих наблюдений, можно заключить, что "БЫСТРОЙ" можно счесть продажу квартиры в период до 3 месяцев, "МЕДЛЕННОЙ" - занявшей больше года*

### <a id="title3.2">3.2. Связь стоимости/цены 1 кв.м и различных параметров квартиры</a>
[к оглавлению](#title11)

**ЧИСЛО КОМНАТ (`rooms`):**

In [ ]:
data.pivot_table(index='rooms', values='last_price', aggfunc='median').reset_index(). \
            plot(x='rooms', y='last_price', grid=True, style='o-', title='Зависимость полной цены от количества комнат', 
            xlabel='количество комнат', ylabel='Цена, руб.', xticks=(range(20))
            );

**Выводы:**
- заметен ожидаемый (близкий к линейному) рост стоимости в зависимости от числа комнат/общей площади (этот параметр изучим внимательней позже)
- заметны экстремальные выбросы стоимости для квартир с 12 и 15 комнатами (несколько эксклюзивных предложений на рынке). 
- ранее на гистограмме распределения предложений по числу комнат видели, что основной массив предложений составляют квартиры с числом комнат до 5, изучим этот интервал подробнее \
Также здесь (и при будущем анализе) включим в исследование зависимости от параметра не только полной цены (стоимости) квартиры, но и цены 1 кв.м (`sqmeter_price`)

In [ ]:
data.query('rooms <= 5').pivot_table(index='rooms', values='last_price', aggfunc='median').reset_index(). \
                plot(x='rooms', y='last_price', grid=True, style='o-',
                 title='Зависимость медианы стоимости от числа комнат', 
                 xlabel='Число комнат', ylabel='Цена, руб.'
                );
plt.show()

data.query('rooms <= 5').pivot_table(index='rooms', values='sqmeter_price', aggfunc='median').reset_index(). \
                plot(x='rooms', y='sqmeter_price', grid=True, style='o-', color='red',
                 title='Зависимость медианы цены 1 кв.м от числа комнат', 
                 xlabel='Число комнат', ylabel='Цена 1 кв.м., руб.'
                );
plt.show()

**Выводы:**
- наблюдаем рост стоимости квартир по мере увеличения числа комнат, по виду зависимость похожа на квадратичную
- стоимость 1 кв. метра максимальна для студий и 1-к квартир, затем существенно (на 8 и 10% соответственно) снижается для 2-3к. квартир, и снова подрастает для 4 и 5к. квартир

**ОБЩАЯ ПЛОЩАДЬ, ЖИЛАЯ ПЛОЩАДЬ, ПЛОЩАДЬ КУХНИ (`total_area`, `living_area`, `kitchen_area`)**  \
(при построении будем изучать диапазон до 99 процентиля, отсекая выбросы вверх (продажи сверхбольших квартир):

In [ ]:
data.query('total_area < total_area.quantile(0.99)').boxplot(column='total_area');
plt.show()
data.query('living_area < living_area.quantile(0.99)').boxplot(column='living_area');
plt.show()
data.query('kitchen_area < kitchen_area.quantile(0.99)').boxplot(column='kitchen_area');

In [ ]:
data.query('total_area <= 160').pivot_table(index='total_area', values='sqmeter_price', aggfunc='median').reset_index(). \
                plot(x='total_area', y='sqmeter_price', grid=True, kind='scatter', alpha=0.4, c='red'). \
                set(title='Зависимость цены 1 кв.м от полной площади', xlabel='полная площадь, кв.м', ylabel='цена 1 кв.м')
plt.ylim([0, 400000])
plt.show();

data.query('living_area <= 110').pivot_table(index='living_area', values='sqmeter_price', aggfunc='median').reset_index(). \
                plot(x='living_area', y='sqmeter_price', grid=True, kind='scatter', alpha=0.4, c='blue'). \
                set(title='Зависимость цены 1 кв.м от жилой площади', xlabel='жилая площадь, кв.м', ylabel='цена 1 кв.м')
plt.ylim([0, 400000])
plt.show();

data.query('kitchen_area <= 25').pivot_table(index='kitchen_area', values='sqmeter_price', aggfunc='median').reset_index(). \
                plot(x='kitchen_area', y='sqmeter_price', grid=True, kind='scatter', alpha=0.4, c='green'). \
                set(title='Зависимость цены 1 кв.м от площади кухни', xlabel='площадь кухни, кв.м', ylabel='цена 1 кв.м')
plt.ylim([0, 400000])
plt.show();

**Выводы:**
- существуют выраженные диапазоны, в которых цена за 1 кв. м. стабильна, колеблется в районе 100 тыс. руб. за 1 кв. м и мало подвержена выбросам: для общей площади это 20-100м. кв., для жилой - 10-75 кв. м, для площади кухни - 5 - 15 кв. м.
- при выходе площади за верхний предел указанных диапазонов нарастает разброс цен (в сторону увеличения)

*Проверим это утверждение, посмотрим таблицу взаимной корреляции и построим матрицу диаграмм рассеяния для тех же площадей и полной цены)*

In [ ]:
data_square = data[['last_price', 'total_area', 'living_area', 'kitchen_area']]. \
    query('total_area < 175 and living_area < 120 and kitchen_area < 35')
data_square.corr()

In [ ]:
pd.plotting.scatter_matrix(data_square, figsize=(9, 9));

**Выводы:**
- практически отсутствует (кажущаяся очевидной по умолчанию) прямая коррелция между площадью кухни и полной/жилой площадью. Это может быть вызвано неаккуратностью в заполнении пункта пользователями, а также тем, что, по мере роста размера квартиры, площадь кухни варьирует в широком диапазоне
- явной корреляции между ценой квартиры и каждым из факторов по отдельности (полная, жилая и площадь кухни) не отмечается
- вероятно, помимо площади на стоимость квартиры влияют и иные факторы, включая расстояние от центра, этажность, день, месяц и год размещения.

**ТИП ЭТАЖА (`floor_type`)**

In [ ]:
data.pivot_table(index='floor_type', values='last_price', aggfunc='median').reset_index(). \
                    plot(kind = 'bar', x='floor_type', y='last_price', grid = True,
                     xlabel='Этаж', ylabel='Полная цена, руб.',
                     title='Зависимость медианы полной цены от этажа'
                    )
plt.xticks([0,1,2], ['первый', 'другой', 'последний'],rotation=0)
plt.ylim([3500000, 5000000])
plt.show()

data.pivot_table(index='floor_type', values='sqmeter_price', aggfunc='median').reset_index(). \
                plot(x='floor_type', y='sqmeter_price', kind = 'bar', grid = True,
                 xlabel='Этаж', ylabel='Цена 1 кв.м, руб.', color='red',
                 title='Зависимость медианы цены 1 кв. м от этажа'
                     )
plt.ylim([70000, 100000])
plt.xticks([0,1,2], ['первый', 'другой', 'последний'],rotation=0)
plt.show()

**Выводы:**
- в квартирах на первом этаже цена 1 кв.м существенно (~ 5-15%!) ниже цены 1 кв.м для квартир на любом другом этаже (возможные причины - шум, загазованность, сравнительно большой риск ограбления и т.п.)
- квартиры на последнем этаже также "теряют" в стоимости по сравнению с квартирами на "других" этажах (возможные причины - отсутствие лифта, протечки, больший риск ограбления и т.п.)

**ГОД РАЗМЕЩЕНИЯ (`publication_year`)**

*Изучим с помощью диаграмм размаха, как полная цена и цена 1 кв.м менялась на протяжении нескольких лет (для 99% значений, исключая эксклюзивные, самые дорогие предложения*

In [ ]:
data.query('last_price < last_price.quantile(0.99)').boxplot(column='last_price', by='publication_year'). \
 set(xlabel='Год размещения объявления', ylabel='Цена, руб.', title="");
plt.show()
data.query('sqmeter_price < sqmeter_price.quantile(0.99)').boxplot(column='sqmeter_price', by='publication_year'). \
 set(xlabel='Год размещения объявления', ylabel='Цена 1 кв. м, руб.', title="");

**Выводы:**
- в 2014 году рынок был стабильнее, с меньшим числом ценовых выбросов (далее надо исследовать его объём, это может быть сопряжено в том числе с малым числом предложений)
- характерное понижение цены 1 кв. метра и полной цены с 2014 заметно на протяжении нескольких лет

*Для наглядности построим графики зависимости медианных значений стоимости и цены 1 кв. м по годам продаж*

In [ ]:
data.query('last_price < last_price.quantile(0.99)'). \
            pivot_table(index='publication_year', values='last_price', aggfunc='median').reset_index(). \
            plot(x='publication_year', y='last_price', grid=True, kind='bar',
                 title='Связь медианы полной цены и года размещения', 
                 xlabel='год размещения', ylabel='цена, руб.'
                )
plt.xticks(rotation=45)
plt.ylim([4000000, 8000000])
plt.show();

data.query('sqmeter_price < sqmeter_price.quantile(0.99)'). \
            pivot_table(index='publication_year', values='sqmeter_price', aggfunc='median').reset_index(). \
            plot(x='publication_year', y='sqmeter_price', grid=True, kind='bar', color='red',
                 title='Связь медианы цены 1 кв.м и года размещения', 
                 xlabel='год размещения', ylabel='цена 1 кв. м, руб.'
                )
plt.ylim([90000, 110000])
plt.xticks(rotation=45)
plt.show();

**Вывод:**
- отчётливо наблюдается провал рынка продаж недвижимости в 2015-2017 годах, восстановление началось только с 2018 года
- медианная цена 1 кв.м восстанавливалась быстрее, чем медиана полной цены

*Дополнительно изучим динамику продаж квартир по годам*

In [ ]:
data['publication_year'].hist(bins=30);

**Вывод:**
- предполагаемая "устойчивость" рынка в 2014 году объясняется в основном малым числом предложений в датафрейме
- наблюдается устойчивый рост числа предложений на протяжении нескольких лет, с 2014 по 2018 включительно, нексмотря даже на "ценовой кризис" 2015-2017 годов
- данные за 2019 год представлены не полностью, исходя из восстановления цены на докризисном уровне можно предположить, что по итогам 2019 года число предложений о продаже может превысить аналогичное в 2018

**ДЕНЬ И МЕСЯЦ РАЗМЕЩЕНИЯ (`publication_week_day`, `publication_month`)**

In [ ]:
data.query('last_price < last_price.quantile(0.99)'). \
            pivot_table(index='publication_week_day', values='last_price', aggfunc='median').reset_index(). \
            plot(x='publication_week_day', y='last_price', grid=True, kind='bar', 
                 title='Связь медианы полной цены и дня размещения', 
                 xlabel='День недели', ylabel='Цена, руб.'
                );
plt.xticks([0,1,2,3,4,5,6], ['пн', 'вт', 'ср', 'чт', 'пт', 'сб', 'вс'], rotation=45)
plt.ylim([4400000, 4800000])
plt.show()

data.query('sqmeter_price < sqmeter_price.quantile(0.99)'). \
            pivot_table(index='publication_week_day', values='sqmeter_price', aggfunc='median').reset_index(). \
            plot(x='publication_week_day', y='sqmeter_price', grid=True, kind='bar', color='red',
                 title='Связь медианы цены 1 кв.м и дня размещения', 
                 xlabel='День недели', ylabel='Цена 1 кв.м, руб.'
                );
plt.ylim([92500, 96000])
plt.xticks([0,1,2,3,4,5,6], ['пн', 'вт', 'ср', 'чт', 'пт', 'сб', 'вс'], rotation=45)
plt.show()

**Вывод:**
- при сопоставимом числе размещений в будние дни, лучшие показатели цены (полной и 1 кв.м) - для объявлений во вторник и среду
- размещаемые в выходные дни (особенно в субботу) объявления по медиане - менее "дорогие"
- разница цены 1 кв.м между "лучшим и "худшим" днями ~ 3%, полной цены ~ 4%

In [ ]:
data.query('last_price < last_price.quantile(0.99)'). \
pivot_table(index='publication_month', values='last_price', aggfunc='median').reset_index(). \
            plot(x='publication_month', y='last_price', grid=True, kind='bar',
                 title='Связь медианы полной цены и месяца размещения', 
                 xlabel='Месяц размещения', ylabel='Цена, руб.'
                );
plt.xticks(range(12), ['янв', 'фев', 'мар', 'апр', 'май', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек'], rotation = 45)
plt.ylim(4300000, 4800000)
plt.show()
data.query('sqmeter_price < sqmeter_price.quantile(0.99)'). \
pivot_table(index='publication_month', values='sqmeter_price', aggfunc='median').reset_index(). \
            plot(x='publication_month', y='sqmeter_price', grid=True, kind='bar', color='red',
                 title='Связь медианы цены 1 кв.м и месяца размещения', 
                 xlabel='Месяц размещения', ylabel='Цена 1 кв.м, руб.'
                );
plt.ylim(92000, 96500)
plt.xticks(range(12), ['янв', 'фев', 'мар', 'апр', 'май', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек'], rotation = 45)
plt.show()

**Вывод:**
- в мае размещается меньше всего объявлений, а в июне, хотя число объявлений и растёт, но размещение идёт с минимальной (по медиане) ценой, как полной, так и 1 кв.м.
- лучший с т.з. размещения месяц - апрель: в топ-3 по числу объявлений и 1 место по медианной цене размещения
- видна чёткая сезонность в размещении - "высокий сезон" февраль-апрель и осень, "низкий сезон" - май-июль и околоновогодние месяцы
- размах цен между "высоким" и "низким" сезонами - в диапазоне от 2,5-3% (цена за 1 кв.м) до 6-8% (полная цена)
- возможно, в "высокий сезон" стараются выставлять на рынок квартиры большей, чем в "низкий" площади

### <a id="title3.3">3.3. Топ-10 населённых пунктов датафрейма по объёмам продаж</a>

[к оглавлению](#title11)

**Исследуем топ-10 населённых пунктов по объёму продаж, изучим в них цену за 1 кв.м**

In [ ]:
#  рассмотрим 10 городов с максимальным числом строк (объявлений)
top_10 = data.query('last_price < last_price.quantile(0.99)').value_counts('locality_name').head(10) 

top_10 = top_10.reset_index()  # переведём наименование города в столбец `locality_name`

In [ ]:
price_1m=[]  # сформируем пустой массив для заполнения расчётными значениями ср.цены 1 кв.м.

#  в цикле заполним этот массив средними значениями цены 1 кв.м. для каждого из городов топ-10
for city in top_10['locality_name']:
    price_1m.append(round(data.loc[data['locality_name'] == city, 'sqmeter_price'].mean()))

top_10['top_mean'] = price_1m  # сделаем получившийся массив дополнительным столбцом `top_mean` в DataFrame `top_10`
top_10.drop([0], axis=1, inplace=True)  #  удалим ненужный теперь столбец с числом объявлений по городам
top_10

In [ ]:
#  фильтруем максимальное и минимальное значения в столбце `top_mean`
top_10.query('top_mean == top_mean.min() or top_mean == top_mean.max()')

**Вывод:**
- вполне ожидаемо максимальная средняя стоимость 1 кв.м оказалась в Санкт_Петербурге
- минимальная средняя стоимость 1 кв. м из крупнейших по числу объявлений городов - в Выборге (в нём же и меньше всего объявлений), это наиболее удалённый и менее развитый из числа сравн. крупных городов Ленобласти
- разница макс. и минимальной средней стоимости 1 кв. м. - почти в 2 раза

### <a id="title3.4">3.4. Связь цены 1 кв.м. и удалённости квартиры от центра Санкт-Петербурга</a>

[к оглавлению](#title11)

*Отфильтруем из общей таблицы строки объявлений из Санкт-Петербурга и создадим сводную таблицу с числом предложений (по каждому км удаления от центра), средней и медианной полной ценой таких предложений:*

In [ ]:
data_spb_full = data.query('locality_name == "Санкт-Петербург"'). \
pivot_table(index='km_to_cityCenter', values='last_price', aggfunc=['mean', 'median', 'count'])

In [ ]:
data_spb_full.columns = ['mean', 'median', 'count']  #  переименуем "двухэтажные" названия столбцов сводной таблицы

In [ ]:
data_spb_full.reset_index(inplace=True) #  сбросим индексы, перенеся значения удаления от центра в отдельный столбец

In [ ]:
data_spb_full = data_spb_full.astype({'km_to_cityCenter':'int'}) #  определим тип данных в этом столбце - integer

*Отфильтруем из общей таблицы объявления из Санкт-Петербурга и создадим сводную таблицу с числом предложений (по каждому км удаления от центра), средней и медианной ценой 1 кв.м. для таких предложений*

In [ ]:
data_spb_sqm = data.query('locality_name == "Санкт-Петербург"'). \
pivot_table(index='km_to_cityCenter', values='sqmeter_price', aggfunc=['mean', 'median', 'count'])

In [ ]:
data_spb_sqm.columns = ['mean', 'median', 'count']  #  переименуем "двухэтажные" названия столбцов сводной таблицы

In [ ]:
data_spb_sqm.reset_index(inplace=True) #  сбросим индексы, перенеся значения удаления от центра в отдельный столбец

In [ ]:
data_spb_sqm = data_spb_sqm.astype({'km_to_cityCenter':'int'}) #  определим данным в этом столбце тип Integer
data_spb_sqm

*Построим графики зависимости медианы/средней цены от расстояния до центра по обеим созданным датафреймам:*

In [ ]:
ax = data_spb_full.plot(kind='bar', x='km_to_cityCenter', y='mean', linewidth=5, alpha=0.5, label='mean', 
                  legend=True, title='Связь средней и медианы полной цены с расстоянием от центра СПб', 
                  ylabel='Полная цена, руб.')
data_spb_full.plot(kind='bar', x='km_to_cityCenter', y='median', linewidth=5, alpha=0.6, label='median', rot=50,
                   ax=ax, color ='red', legend=True, xlabel='Расстояние от центра, км', figsize=(9,6))
plt.show()

bx = data_spb_sqm.plot(kind='bar', x='km_to_cityCenter', y='mean', linewidth=5, alpha=0.7, label='mean', 
                  legend=True, title='Связь средней и медианы цены за 1 кв.м. с расстоянием от центра СПб', 
                  ylabel='Цена 1 кв.м, руб.', color ='yellow')
data_spb_sqm.plot(kind='bar', x='km_to_cityCenter', y='median', linewidth=5, alpha=0.6, label='median', rot=50,
                   ax=bx, color ='green', legend=True, xlabel='Расстояние от центра, км', figsize=(9,6))
plt.show()

**Вывод:**
- видна тенденция плавного снижения стоимости квартир и цены 1 кв.м по мере удаления от центра СПб 
- самые дорогие в датафрейме - квартиры расположенные вплотную (0 км) к центру
- для расстояний 1-3 км стоимость и цена 1 кв.м сравнительно быстро понижаются, возможно это связано с изношенностью жилого фонда + необходимостью для покупателя вкладываться в недешёвый ремонт
- спад стоимости на при этом происходит быстрее, чем цены 1 кв.м, возможно, из этого следует, что в среднем площадь продаваемых квартир по мере удаления от центра снижается
- затем, в радиусе от 4 до 8 км, цены подрастают, после чего продолжают снижение, видимо, мы можем корректно оценить понятие "квартира в центре" для СПб - как квартира в радиусе до 8 км от центра Санкт-Петербурга
- всплеск цены в сильно удалённых от центра объектах (27-29 км) связан во-первых с единичным числом данных для усреднения (что заметно повышает вклад цены отдельного предложения), а во-вторых - с возможной близостью таких квартир к неким объектам, которые могут повысить стоимость покупки (пляж, загородные дворцы, известные парки и т.п.) уже вне связи с расстоянием до центра
- заметно существенное расхождение mean и median в ближних к центру районах, что можно объяснить выбросами, когда на бОльшую цену влияет не только удалённость непосредственно от центра, но и близость к тем или иным объектам (театры, музеи, парки и т.п.), по мере удаления от центра (8-9 км) расхождение (соответственно и влияние выбросов) заметно снижаются

# <a id="title4">Общий вывод:</a>

[к оглавлению](#title11)

- представленный датафрейм содержит сводные данные по рынку недвижимости СПб и ЛенОбласти за 4 года (конец ноября 2014 - начало мая 2019), основан на заполненных пользователями анкетах/объявлениях с параметрами квартир + дополнительными картографическими данными
- целью проекта являлось исследование представленного датафрейма, обработка и подготовка данных к анализу, устранение дубликатов, локализация и по возможности исправление аномальных и недостающих значений в данных, а также последующий анализ - изучение максимально влияющих на цену недвижимости факторов (площади, этажности объекта, его удалённости от центра и т.п.) 


 - в рамках предобработки:
 1. Заменили пропуски в данных медианными значениями (там, где это было возможно), ориентируясь на общность ряда характеристик для конкретных населённых пунктов
 2. Обнаружили, что если для пользовательских данных подобная замена возможна (с ограничениями), то для картографических данных её произвести невозможно, в связи с недостатком информации и отсутствием источников для восполнения пропусков
 3. В связи с тем, что при дальнейшем анализе картографические данные  практически не использовались, оставили строки, содержащие в них пропуски, в датафрейме
 4. Используя доступную в интернете информацию для СПб и Лен. области (о стандартных высотах, площадях квартир, этажности и т.п.), произвели коррекцию данных и удаление аномальных значений в пользовательских данных
 5. В ряде случаев, в связи с невозможностью коррекции = удалили строки с аномалиями из датафрейма (не более 1% от общего массива данных) 


 - после этого приступили к анализу, результатом которого стали несколько наблюдений:
 1. Основной массив представленной в датафрейме недвижимости - квартиры, общей площадью до 75 кв. метров, с 1-3 комнатами, жилой площадью до 50 кв.м и кухнями 10 кв.м и стандартными потолками 2-4-2.7 м., что соответствует массовой застройке советских времён \
 1а. Наряду с этим встречаются и эксклюзивные предложения, с многосотметровыми квартирами (возможно, досоветской постройки)
 2. Прямой зависимости стоимости жилья (1 кв.м) от таких факторов, как общая/жилая площадь и размеры кухни не обнаружено, уровень цены колебался на протяжении исследуемого периода в пределах 90-95 тыс. руб. (без поправки на инфляцию за эти годы) \
 - 2а. Ярко выражена зависимость стоимости квартир от расположения: нижние этажи заметно (на единицы процентов) дешевле других, верхние этажи также продаются с дисконтом по сравнению со "средними" \
 - 2б. Наблюдается также некоторая связь между стоимостью/ценой и размещением объявления в определённый день и месяц (размещённые в субботу объявления как правило с более низкой ценой, чем в любой из будних дней; в течение года выражена сезонность - после "дорогого" апреля в мае-июне наблдается спад предложений и снижение цен. В обоих примерах речь идёт о единицах процентов \
  - 2в. Отчётливо наблюдался кризис на рынке недвижимости с удешевлением квартир в 2015-217 годах и восстановительный рост в 2018 - начале 2019 \
  - 2г. Цена 1 кв.м наиболее велика в студиях и 1к квартирах, ниже всего - в 3 к.квартирах, после чего с увеличением числа комнат начинается рост цены, что можно объяснить разным "уровнем" квартир (до 3к. - "стандартное" жильё, выше - "бизнес" или "люкс" классы), рост продолжается до числа комнат в 7, после чего уровень цен стабилизируется. Разница между "наибольшей" и "наименьшей" ценой составляет заметную величину ~ 10%
 3. Были исследованы цены на 1 кв.м. в крупнейших городах, в лидерах, как и ожидалось, оказался Санкт-Петербург, как город с максимальным предложением разнообразной, включая престижную, недвидимости. Замыкает десятку городов Выборг, с почти 2х кратным отставанием по цене 1 кв.м от лидера списка.
 4. Была проверена зависимость цены и стоимости от расстояния до центра Санкт-Петербурга, выяснилось, что в ближайшей окрестности центра (до 1 км) предлагались максимальные по цене варианты, также высоко в исследуемый период ценились квартиры в радиусе 3-8 км (что можно считать "границей" для центра Санкт-Петербурга). В пределах центра СПб наблюджаются существеные выбросы в ценах квартир, когда более высокая цена (в пересчётек на 1 кв.м)  связана не только с расстоянием, но с расположением относительно значимых "центральных" объектов города